In [ ]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
                    .appName('helloSpark')
                    .getOrCreate()
        )

Further info on Spark sessions:  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/spark_session.html

In [ ]:
spark

In [ ]:
# Let's generate some data for analysis

import random

names = ["Alice", "Ben", "Charles", "Daisy"]
start_range = 900
end_range = 5000
python_data = [[random.choice(names),random.randint(start_range,end_range)] for i in range(500000)]

In [ ]:
# To read in a Python object (list, dict), we can use spark.createDataFrame
# We define a schema to have nicer column names and avoid Spark having to infer the schema
schema = "name STRING, salary INT"

df = spark.createDataFrame(python_data,schema=schema)

In [ ]:
# to display some rows, you can use .show() 
df.show()

In [ ]:
df_new = (df.groupBy("name")
          .avg("salary")
         )

In [ ]:
df_new.show()

In [ ]:
# Many of the functions hide behind spark.sql.functions
import pyspark.sql.functions as F

(df_new.select(
    "name"
    ,"avg(salary)"
    ,F.round("avg(salary)").alias("average")
    ).show()
)

The following is for a comparison with the popular Python package `pandas`

In [ ]:
import pandas as pd 

pd_df = pd.DataFrame(python_data,columns=["name","salary"])

In [ ]:
pd_df

In [ ]:
pd_df.groupby("name").mean("salary")

Which of these seemed to be faster?  
Why?

Let's have a quick walkthrough of a few more PySpark methods.  
For a longer (full) list of methods, see:  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html

In [ ]:
# load CSV files into DataFrames
employees_df = spark.read.option("header", "true").csv("input/employees.csv")
departments_df = spark.read.option("header", "true").csv("input/departments.csv")

employees_df.show()
departments_df.show()

In [ ]:

# convert salary from string to integer for proper filtering
employees_df = employees_df.withColumn("salary", F.col("salary").cast("integer"))

employees_df.show()

In [ ]:

# use .filter() for ... filtering.
filtered_df = employees_df.filter(F.col("salary") > 55000)

# use .select() for ... selecting (columns).
selected_df = filtered_df.select("emp_id", "name", "department", "salary")

# use withColumnRenamed for renaming columns
renamed_df = selected_df.withColumnRenamed("emp_id", "employee_id")

# use selectExpr() for projecting SQL expressions
expr_df = renamed_df.selectExpr("employee_id", "name", "department", "salary", "salary / 12 as monthly_salary")

# use .join() for ... joining. Let's join with departments DataFrame on the 'department' column
joined_df = expr_df.join(departments_df, on="department", how="inner")


In [ ]:

# .write for writing. There are multiple more options you can see in the next classes.
joined_df.write.mode("overwrite").option("header", "true").csv("output/joined_employees")

# NB - Spark has lazy evaluation. It will only execute the code when it needs to.
# This means that you can chain multiple transformations and actions together without any performance hit.
# The code will only be executed when you call an action like .show() or .write().